In [1]:
import numpy as np
import tensorflow as tf
import pickle
from tensorflow.keras import layers , activations , models
import requests, zipfile, io
from tensorflow.keras import preprocessing , utils
import os
import yaml
import string
import re

In [ ]:
r = requests.get( 'https://github.com/shubham0204/Dataset_Archives/blob/master/chatbot_nlp.zip?raw=true' ) 
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall()

In [32]:
dir_path = 'chatbot_nlp/data'
# files_list = os.listdir(dir_path + os.sep)
a = ["hi", "oops"]

for i in a[1:]:
    print(i)

oops


In [2]:
dir_path = 'chatbot_nlp/data'
files_list = os.listdir(dir_path + os.sep)

questions = list()
answers = list()

for filepath in files_list:
    stream = open( dir_path + os.sep + filepath , 'rb')
    docs = yaml.safe_load(stream)
    conversations = docs['conversations']
    for con in conversations:
        if len( con ) > 2 :
            questions.append(con[0])
            replies = con[ 1 : ]
            ans = ''
            for rep in replies:
                ans += ' ' + rep
            answers.append( ans )
        elif len( con )> 1:
            questions.append(con[0])
            answers.append(con[1])
# print(questions)
# print(answers)
answers_with_tags = list()
for i in range( len( answers ) ):
    if type( answers[i] ) == str:
        answers_with_tags.append( answers[i] )
    else:
        print('f')
        questions.pop( i )

answers = list()
for i in range( len( answers_with_tags ) ) :
    answers.append( '<START> ' + answers_with_tags[i] + ' <END>' )

tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts( questions + answers )
VOCAB_SIZE = len( tokenizer.word_index )+1
print( 'VOCAB SIZE : {}'.format( VOCAB_SIZE ))


f
f
VOCAB SIZE : 1894


In [3]:
# encoder_input_data
tokenized_questions = tokenizer.texts_to_sequences( questions )
maxlen_questions = max( [ len(x) for x in tokenized_questions ] )
padded_questions = preprocessing.sequence.pad_sequences( tokenized_questions , maxlen=maxlen_questions , padding='post' )
encoder_input_data = np.array( padded_questions )
print( encoder_input_data.shape , maxlen_questions )

# decoder_input_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
maxlen_answers = max( [ len(x) for x in tokenized_answers ] )
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
decoder_input_data = np.array( padded_answers )
print( decoder_input_data.shape , maxlen_answers )

# decoder_output_data
tokenized_answers = tokenizer.texts_to_sequences( answers )
for i in range(len(tokenized_answers)) :
    tokenized_answers[i] = tokenized_answers[i][1:]
padded_answers = preprocessing.sequence.pad_sequences( tokenized_answers , maxlen=maxlen_answers , padding='post' )
onehot_answers = utils.to_categorical( padded_answers , VOCAB_SIZE )
decoder_output_data = np.array( onehot_answers )
print( decoder_output_data.shape )

# Saving all the arrays to storage
np.save( './data/enc_in_data.npy' , encoder_input_data )
np.save( './data/dec_in_data.npy' , decoder_input_data )
np.save( './data/dec_tar_data.npy' , decoder_output_data )

(564, 22) 22
(564, 74) 74
(564, 74, 1894)


In [4]:
# Training Model

encoder_inputs = tf.keras.layers.Input(shape=( None , ))
encoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True ) (encoder_inputs)
encoder_outputs , state_h1 , state_c1 = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_embedding )
# encoder_outputs , state_h2 , state_c2 = tf.keras.layers.LSTM( 200 , return_state=True )( encoder_outputs1 )
# encoder_states = [ state_h1 , state_c1 , state_h2 , state_c2 ]
encoder_states = [ state_h1 , state_c1]

decoder_inputs = tf.keras.layers.Input(shape=( None ,  ))
decoder_embedding = tf.keras.layers.Embedding( VOCAB_SIZE, 200 , mask_zero=True) (decoder_inputs)
decoder_lstm1 = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
# decoder_lstm2 = tf.keras.layers.LSTM( 200 , return_state=True , return_sequences=True )
decoder_outputs , _ , _ = decoder_lstm1 ( decoder_embedding , initial_state=encoder_states[0:2] )
# decoder_outputs , _ , _ = decoder_lstm2 ( decoder_outputs , initial_state=encoder_states[2:4] )
decoder_dense = tf.keras.layers.Dense( VOCAB_SIZE , activation=tf.keras.activations.softmax ) 
output = decoder_dense ( decoder_outputs )

model = tf.keras.models.Model([encoder_inputs, decoder_inputs], output )
model.compile(optimizer=tf.keras.optimizers.RMSprop(), loss='categorical_crossentropy')

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 200)    378800      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, None, 200)    378800      input_2[0][0]                    
______________________________________________________________________________________________

In [ ]:
embedding_matrix = np.load('./data/embedding_matrix.npy')

embedding = layers.Embedding(vocab_size, 200, weights=[embedding_matrix], trainable=False)

encoder_input = layers.Input(shape=(None, ))
encoder_lstm = layers.LSTM(200, return_state=True)
_ , state_h1, state_c1 = encoder_lstm(embedding(encoder_input))

decoder_input = layers.Input(shape=(None, ))
decoder_lstm = layers.LSTM(200, return_state=True, return_sequences=True)

decoder_output, _, _ = decoder_lstm(embedding(decoder_input), initial_state=[state_h1, state_c1])
decoder_dense = layers.Dense(vocab_size, activation='softmax')
output = decoder_dense(decoder_output)

In [12]:
model.fit([encoder_input_data , decoder_input_data], decoder_output_data, batch_size=50, epochs=25 ) 
model.save( 'model.h5' ) 

Train on 564 samples
Epoch 1/25
564/564 [==============================] - 1s 1ms/sample - loss: 0.3131
Epoch 2/25
564/564 [==============================] - 1s 978us/sample - loss: 0.3056
Epoch 3/25
564/564 [==============================] - 1s 967us/sample - loss: 0.2984
Epoch 4/25
564/564 [==============================] - 1s 970us/sample - loss: 0.2934
Epoch 5/25
564/564 [==============================] - 1s 956us/sample - loss: 0.2867
Epoch 6/25
564/564 [==============================] - 1s 1ms/sample - loss: 0.2797
Epoch 7/25
564/564 [==============================] - 1s 953us/sample - loss: 0.2738
Epoch 8/25
564/564 [==============================] - 1s 950us/sample - loss: 0.2669
Epoch 9/25
564/564 [==============================] - 1s 964us/sample - loss: 0.2595
Epoch 10/25
564/564 [==============================] - 1s 960us/sample - loss: 0.2565
Epoch 11/25
564/564 [==============================] - 1s 958us/sample - loss: 0.2494
Epoch 12/25
564/564 [=========================

In [9]:
def make_inference_models():
    
    encoder_model = tf.keras.models.Model(encoder_inputs, encoder_states)
    
    decoder_state_input_h = tf.keras.layers.Input(shape=( 200 ,))
    decoder_state_input_c = tf.keras.layers.Input(shape=( 200 ,))
    
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    
    decoder_outputs, state_h, state_c = decoder_lstm1(
        decoder_embedding , initial_state=decoder_states_inputs)
    decoder_states = [state_h, state_c]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = tf.keras.models.Model(
        [decoder_inputs] + decoder_states_inputs,
        [decoder_outputs] + decoder_states)
    
    return encoder_model , decoder_model

In [10]:
def str_to_tokens( sentence : str ):
    translator = str.maketrans('', '', string.punctuation)
    words = sentence.lower().translate(translator).split()
    
    tokens_list = list()
    for word in words:
        try:
            a = tokenizer.word_index[ word ]
        except KeyError:
            a = 2
        tokens_list.append( a ) 
    return preprocessing.sequence.pad_sequences( [tokens_list] , maxlen=maxlen_questions , padding='post')

In [13]:
enc_model , dec_model = make_inference_models()

for _ in range(10):
    states_values = enc_model.predict( str_to_tokens( input( 'Enter question : ' ) ) )
    empty_target_seq = np.zeros( ( 1 , 1 ) )
    empty_target_seq[0, 0] = tokenizer.word_index['start']
    stop_condition = False
    decoded_translation = ''
    while not stop_condition :
        dec_outputs , h , c = dec_model.predict([ empty_target_seq ] + states_values )
        sampled_word_index = np.argmax( dec_outputs[0, -1, :] )
        sampled_word = None
        for word , index in tokenizer.word_index.items() :
            if sampled_word_index == index :
                decoded_translation += ' {}'.format( word )
                sampled_word = word
        
        if sampled_word == 'end' or len(decoded_translation.split()) > maxlen_answers:
            stop_condition = True
            
        empty_target_seq = np.zeros( ( 1 , 1 ) )  
        empty_target_seq[ 0 , 0 ] = sampled_word_index
        states_values = [ h , c ] 

    print( decoded_translation )


Enter question :  hello


 hi end


Enter question :  how are you


 i am not to be for your own chat end


Enter question :  why is that


...

 hal misses sal end


Enter question :  bad


 hello end


Enter question :  hello


 hi end


In [27]:
# print(tokenizer.word_index)
# print(questions[0:2])
# print(answers[2])
# print(tokenized_answers[0])
# print(str_to_tokens("neural"))
# print(len(questions))
# print(len(answers))

regex = '(?<=[^ ])(?=[.,!?()])|(?<=[.,!?()])(?=[^ ])'
s = 'Hello. mY boiE!'
print(s)
print(re.sub(regex, r' ', s))

Hello. mY boiE!
Hello . mY boiE !


In [29]:
embedding_matrix = np.load('./data/embedding_matrix.npy')
print(embedding_matrix.shape)

(1912, 200)
